_________________
_________________
_________________
_________________

## Reading data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# supress futurewarning
# https://stackoverflow.com/questions/32612180/eliminating-warnings-from-scikit-learn

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix, classification_report, confusion_matrix

from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

In [4]:
from q_neighbors import QNN, show_figure, drop_duplicates_and_correct_ambiguities

In [5]:
from os import listdir
from os.path import isfile, join

In [6]:
# train data

# .loc[:, 1:] so that first column (g values) is dropped
df_train = pd.read_csv("./data_new2/train_1000/j00_train.csv", header=None).loc[:, 1:]

df_train.columns = ["y"] + [f"X_{i}" for i in range(1, df_train.shape[1])]

df_train

,y,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18
0,0,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000,0.000000,0.000000e+00,1.000000
1,0,0.000002,-5.000005e-07,0.999999,0.000001,-2.432937e-13,0.999999,0.000001,5.868290e-15,0.999999,0.000001,5.868359e-15,0.999999,0.000001,5.868567e-15,0.999999,0.000001,5.868584e-15,0.999999
2,0,0.000006,-2.000008e-06,0.999996,0.000004,-3.988789e-12,0.999996,0.000004,1.418562e-14,0.999996,0.000004,1.418377e-14,0.999996,0.000004,1.418362e-14,0.999996,0.000004,1.418455e-14,0.999996
3,0,0.000014,-4.500040e-06,0.999991,0.000009,-2.024686e-11,0.999991,0.000009,3.048842e-15,0.999991,0.000009,3.033148e-15,0.999991,0.000009,3.032688e-15,0.999991,0.000009,3.027786e-15,0.999991
4,0,0.000024,-8.000128e-06,0.999984,0.000016,-6.398412e-11,0.999984,0.000016,1.514318e-14,0.999984,0.000016,1.859920e-14,0.999984,0.000016,1.859908e-14,0.999984,0.000016,1.859911e-14,0.999984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1,0.930341,-2.258643e-01,0.259848,0.874498,-2.737551e-02,0.099188,0.871963,-6.763380e-03,0.042074,0.871703,-2.136313e-03,0.019219,0.871667,-8.371226e-04,0.010114,0.871663,-5.465156e-04,0.007674
997,1,0.930496,-2.256914e-01,0.259568,0.874750,-2.732923e-02,0.098977,0.872224,-6.745370e-03,0.041940,0.871965,-2.128406e-03,0.019136,0.871930,-8.330590e-04,0.010059,0.871926,-5.434626e-04,0.007627
998,1,0.930650,-2.255187e-01,0.259287,0.875001,-2.728306e-02,0.098767,0.872485,-6.727420e-03,0.041807,0.872227,-2.120536e-03,0.019054,0.872193,-8.290187e-04,0.010004,0.872188,-5.404291e-04,0.007581
999,1,0.930804,-2.253462e-01,0.259008,0.875252,-2.723699e-02,0.098558,0.872744,-6.709531e-03,0.041674,0.872489,-2.112700e-03,0.018972,0.872454,-8.250014e-04,0.009949,0.872449,-5.374150e-04,0.007534


_________

In [7]:
# reading test data into dictionary
# applying same transfs applied to training data

path_test = "./data_new2/test_1000"
files = [f"{path_test}/{f}" for f in listdir(path_test) if isfile(join(path_test, f))]

# keeping the first collumn (ID)
df_test_dic = {f.split("/")[-1][:-4] : pd.read_csv(f, header=None) for f in files}

for k, v in df_test_dic.items():
    
    v.columns = ["g"] + [f"X_{i}" for i in range(1, v.shape[1])]
        
    print(f"\nShape of test data {k}")
    print(v.shape)
    
    df_test_dic[k] = v

    print("\n")
    print("="*60)


Shape of test data j00
(1001, 19)



Shape of test data j01
(1001, 19)



Shape of test data j02
(1001, 19)



Shape of test data j03
(1001, 19)



Shape of test data j04
(1001, 19)



Shape of test data j05
(1001, 19)



Shape of test data j06
(1001, 19)



Shape of test data j07
(1001, 19)



Shape of test data j08
(1001, 19)



Shape of test data j09
(1001, 19)



Shape of test data j10
(1001, 19)




________

______

## Training data preprocessing

- 1 - dimensionality reduction;
- 2 - binarization.

_____

## 1 - dimensionality reduction with PCA

In [ ]:
X = df_train.drop(columns = "y")
y = df_train["y"]

#____________________________________

ss = StandardScaler().fit(X)

X_ss = pd.DataFrame(ss.transform(X), index=X.index, columns=X.columns)

#____________________________________

In [ ]:
from sklearn.decomposition import PCA

pca2 = PCA(n_components=2).fit(X_ss)
pca3 = PCA(n_components=3).fit(X_ss)
pca4 = PCA(n_components=4).fit(X_ss)

pca_dict = {"2-PCs" : pca2,
            "3-PCs" : pca3,
            "4-PCs" : pca4}

In [ ]:
import os

def model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy):   
    
    for strategy, most_important_features in pca_dict.items():

        print()
        print("#"*80)
        print(f"Feature selection strategy: {strategy}".center(80))
        print("#"*80)
        print()

        for n_bins, n_feats in combs:

            ###################################################

            file_specs = f"{n_feats}_{n_bins}_desambiguation_{how}_pca_{strategy}_discretization_{kbins_strategy}_refit_discret"

            ###################################################

            start = time.time()

            print("\n")
            print("="*50)
            print(f"n_bins : {n_bins}".center(50))
            print(f"n_feats : {n_feats}".center(50))
            print("="*50)
            print("\n")

            ###################################################
            
            X_pca = pd.DataFrame(pca_dict[f"{n_feats}-PCs"].transform(X_ss),
                                 columns = [f'pc{i+1}' for i in range(n_feats)])

            # creating "n_bins" uniform bins to discretize continuous features
            bin_pca = KBinsDiscretizer(n_bins=n_bins, 
                                       encode='ordinal',
                                       strategy=kbins_strategy).fit(X_pca)

            # transforming feature importance-reduced dataset
            aux = pd.DataFrame(bin_pca.transform(X_pca)).astype("str")
            aux.columns = [f"Xb_{i+1}" for i in range(aux.shape[1])]

            # creating dummy variables
            aux = pd.get_dummies(aux, drop_first=True)

            df_pca_bin = pd.concat([aux, y], axis=1)

            ###################################################

            X_pca_bin = df_pca_bin.drop(columns="y")
            y_pca_bin = df_pca_bin["y"]

            ###################################################

            # desambiguation is made within .fit() method!
            qnn_pca_bin = QNN().fit(X_pca_bin, y_pca_bin, how=how)

            ###################################################

            pipe = Pipeline([('knn', KNeighborsClassifier(n_jobs=-1))])

            param_grid = {'knn__n_neighbors': np.arange(1, 100),
                          'knn__weights':['uniform', 'distance'], 
                          'knn__metric':['euclidean', 'hamming']}

            # with verbose=10, info appears in the terminal/prompt!
            knn_pca_bin = GridSearchCV(pipe, param_grid, scoring="f1_weighted", 
                                       cv=10, verbose=10, n_jobs=-1)

            knn_pca_bin.fit(X_pca_bin.values, y_pca_bin)

            print(f"\nBest classical model parameters:\n{knn_pca_bin.best_params_}")

            ###################################################

            print("\n\n")

            for label, df_test in df_test_dic.items():
                
                # ========================================
                
                file_quantum = f"./results/predict_proba_test_{label}_QUANTUM_pca_new_features_{file_specs}.csv"
                file_classical = f"./results/predict_proba_test_{label}_CLASSICAL_pca_new_features_{file_specs}.csv"
                
                # ========================================

                if os.path.isfile(file_quantum):
                    
                    print(f"File with predictions of {label} (QUANTUM) already exists!\n")
                    
                else:
                    
                    print(f"Predicting targets for {label} with feature importance - QUANTUM!\n")

                    X_test = df_test.drop(columns=["g"]).copy()

                    # ========================================

                    # new 01/04/2022: refit ss in test
                    ss_test = StandardScaler()
                    X_test_ss = pd.DataFrame(ss_test.fit_transform(X_test), index=X_test.index, columns=X_test.columns)
                    
                    X_test_pca = pd.DataFrame(pca_dict[f"{n_feats}-PCs"].transform(X_test_ss),
                                              columns = [f'pc{i+1}' for i in range(n_feats)])
                    
                    # new 25/03/22: refit bin in test
                    bin_pca_test = KBinsDiscretizer(n_bins=n_bins, encode='ordinal',strategy=kbins_strategy)
                    aux = pd.DataFrame(bin_pca_test.fit_transform(X_test_pca)).astype("str")
                    aux.columns = [f"Xb_{i+1}" for i in range(aux.shape[1])]

                    X_test_pca = pd.get_dummies(aux, drop_first=True)

                    # ========================================

                    y_pred_pca, y_proba_pca = qnn_pca_bin.predict(X_test_pca)

                    # ========================================

                    df_result = pd.concat([df_test["g"], 
                                           pd.Series(np.array(y_proba_pca)[:, 1], name="predict_proba_y=1")], 
                                          axis=1)

                    df_result.to_csv(file_quantum, index=False)

                ###################################################    
                # CLASSICAL BENCHMARK
                ###################################################    

                if os.path.isfile(file_classical):
                    
                    print(f"File with predictions of {label} (CLASSICAL) already exists!\n")
                    
                else:
                    print(f"Predicting targets for {label} with feature importance - CLASSICAL!")

                    # ========================================

                    X_test_pca = X_test_pca.to_numpy()

                    train_dims = X_pca_bin.shape[1]
                    test_dims = X_test_pca.shape[1]

                    # in this case, i'll just try some padding
                    if test_dims < train_dims:

                        X_test_pca = np.pad(X_test_pca, 
                                            [(0, 0), (0, train_dims - test_dims)],
                                            mode='constant', 
                                            constant_values=0).copy()

                    # in this case, there's not much to be done...
                    # but i think this won't happen. let's cover it anyway, though
                    elif test_dims > train_dims:

                        error_str = "Dimensionality mismatch!"
                        error_str += f"\nTraining data has {X_pca_bin.shape[1]} dimensions"
                        error_str += f"\nAnd test dat has {X_test_pca.shape[1]} dimensions!"
                        error_str += f"\nPlease check pre-processing!"

                        raise ValueError(error_str)

                    # ========================================

                    y_proba_pca = knn_pca_bin.predict_proba(X_test_pca)

                    # ========================================

                    df_result = pd.concat([df_test["g"], 
                                           pd.Series(np.array(y_proba_pca)[:, 1], name="predict_proba_y=1")], 
                                          axis=1)

                    df_result.to_csv(file_classical, index=False)

                    print("\n")
                    print("~"*30)
                    print("\n")

            ###################################################

            stop = time.time()
            duration = time.strftime("%H:%M:%S", time.gmtime(stop-start))

            print(f"\n----> Model execution time: {duration}")

Combinations of number of features and number of bins to be tested

In [ ]:
n_bins_list = [2, 3]
n_feats_list = [2, 3, 4]

combs = [(2, 2), 
         (2, 3), (2, 4),
         (3, 3), (3, 4)]

### No desambiguation

In [ ]:
how = None

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "quantile")

### Desambiguation with minority class

In [ ]:
how = "minor"

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "quantile")

### Desambiguation with majority class

In [ ]:
how = "major" 

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "quantile")

### Desambiguation with first appearance class

In [ ]:
how = "first"

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, pca_dict, combs, kbins_strategy = "quantile")

_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______

_____

## 2 - dimensionality reduction with feature importance

In [8]:
X = df_train.drop(columns = "y")
y = df_train["y"]

#____________________________________

ss = StandardScaler().fit(X)

X_ss = pd.DataFrame(ss.transform(X), index=X.index, columns=X.columns)

#____________________________________

In [14]:
def feature_imp_stats(X_ss, y, n_feats_list = [2, 3, 4], n_runs = 1000, kind = "extra"):

    important_features = {k : {} for k in n_feats_list}

    for seed in range(n_runs):
        
        print(f"\nSeed: {seed}\n")
        
        if kind == "extra":
            
            forest = ExtraTreesClassifier(n_estimators=1000, random_state=seed, n_jobs=-1)
            
        elif kind == "random":
            
            forest = RandomForestClassifier(n_estimators=1000, random_state=seed, n_jobs=-1)

        forest.fit(X_ss, y)

        importances = forest.feature_importances_
        indices_fi = np.argsort(importances)[::-1]
        
        for n_feats in n_feats_list:

            X_fi = X_ss.iloc[:, indices_fi[:n_feats]].copy()

            most_imp = X_ss.columns[indices_fi[:n_feats]]

            print(f"{n_feats} most important features:\t{most_imp}")

            for feature in most_imp:

                if feature not in important_features[n_feats]:

                    important_features[n_feats][feature] = 1

                else:

                    important_features[n_feats][feature] += 1
                
    # ordered by values
    return {k: dict(sorted(v.items(), key=lambda item: item[1], reverse=True)) for k, v in important_features.items()}

### Feature importance

Execution of cells below is blocked (frozen) because they take a long time to run! Also, they're only used to determine the features to be used, which are fixed below in the code. So, we don't have to run these cells every time.

But, if you want to run them, just unfreeze (nb extensions is required).

In [10]:
important_features_random = feature_imp_stats(X_ss, y, 
                                              n_feats_list = [2, 3, 4], n_runs = 1000, kind="random")


Seed: 0

2 most important features:	Index(['X_4', 'X_18'], dtype='object')
3 most important features:	Index(['X_4', 'X_18', 'X_2'], dtype='object')
4 most important features:	Index(['X_4', 'X_18', 'X_2', 'X_16'], dtype='object')

Seed: 1

2 most important features:	Index(['X_12', 'X_18'], dtype='object')
3 most important features:	Index(['X_12', 'X_18', 'X_7'], dtype='object')
4 most important features:	Index(['X_12', 'X_18', 'X_7', 'X_15'], dtype='object')

Seed: 2

2 most important features:	Index(['X_12', 'X_13'], dtype='object')
3 most important features:	Index(['X_12', 'X_13', 'X_16'], dtype='object')
4 most important features:	Index(['X_12', 'X_13', 'X_16', 'X_15'], dtype='object')

Seed: 3

2 most important features:	Index(['X_2', 'X_18'], dtype='object')
3 most important features:	Index(['X_2', 'X_18', 'X_3'], dtype='object')
4 most important features:	Index(['X_2', 'X_18', 'X_3', 'X_7'], dtype='object')

Seed: 4

2 most important features:	Index(['X_2', 'X_9'], dtype='object'

2 most important features:	Index(['X_7', 'X_15'], dtype='object')
3 most important features:	Index(['X_7', 'X_15', 'X_4'], dtype='object')
4 most important features:	Index(['X_7', 'X_15', 'X_4', 'X_16'], dtype='object')

Seed: 37

2 most important features:	Index(['X_16', 'X_10'], dtype='object')
3 most important features:	Index(['X_16', 'X_10', 'X_15'], dtype='object')
4 most important features:	Index(['X_16', 'X_10', 'X_15', 'X_1'], dtype='object')

Seed: 38

2 most important features:	Index(['X_12', 'X_2'], dtype='object')
3 most important features:	Index(['X_12', 'X_2', 'X_9'], dtype='object')
4 most important features:	Index(['X_12', 'X_2', 'X_9', 'X_10'], dtype='object')

Seed: 39

2 most important features:	Index(['X_16', 'X_15'], dtype='object')
3 most important features:	Index(['X_16', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_16', 'X_15', 'X_9', 'X_12'], dtype='object')

Seed: 40

2 most important features:	Index(['X_18', 'X_16'], dtype='object')
3 

2 most important features:	Index(['X_18', 'X_10'], dtype='object')
3 most important features:	Index(['X_18', 'X_10', 'X_15'], dtype='object')
4 most important features:	Index(['X_18', 'X_10', 'X_15', 'X_16'], dtype='object')

Seed: 73

2 most important features:	Index(['X_3', 'X_12'], dtype='object')
3 most important features:	Index(['X_3', 'X_12', 'X_4'], dtype='object')
4 most important features:	Index(['X_3', 'X_12', 'X_4', 'X_13'], dtype='object')

Seed: 74

2 most important features:	Index(['X_10', 'X_15'], dtype='object')
3 most important features:	Index(['X_10', 'X_15', 'X_4'], dtype='object')
4 most important features:	Index(['X_10', 'X_15', 'X_4', 'X_13'], dtype='object')

Seed: 75

2 most important features:	Index(['X_7', 'X_6'], dtype='object')
3 most important features:	Index(['X_7', 'X_6', 'X_15'], dtype='object')
4 most important features:	Index(['X_7', 'X_6', 'X_15', 'X_10'], dtype='object')

Seed: 76

2 most important features:	Index(['X_7', 'X_10'], dtype='object')
3 m

2 most important features:	Index(['X_18', 'X_10'], dtype='object')
3 most important features:	Index(['X_18', 'X_10', 'X_9'], dtype='object')
4 most important features:	Index(['X_18', 'X_10', 'X_9', 'X_16'], dtype='object')

Seed: 109

2 most important features:	Index(['X_2', 'X_12'], dtype='object')
3 most important features:	Index(['X_2', 'X_12', 'X_4'], dtype='object')
4 most important features:	Index(['X_2', 'X_12', 'X_4', 'X_6'], dtype='object')

Seed: 110

2 most important features:	Index(['X_15', 'X_6'], dtype='object')
3 most important features:	Index(['X_15', 'X_6', 'X_3'], dtype='object')
4 most important features:	Index(['X_15', 'X_6', 'X_3', 'X_4'], dtype='object')

Seed: 111

2 most important features:	Index(['X_10', 'X_4'], dtype='object')
3 most important features:	Index(['X_10', 'X_4', 'X_3'], dtype='object')
4 most important features:	Index(['X_10', 'X_4', 'X_3', 'X_13'], dtype='object')

Seed: 112

2 most important features:	Index(['X_18', 'X_2'], dtype='object')
3 mos

2 most important features:	Index(['X_16', 'X_4'], dtype='object')
3 most important features:	Index(['X_16', 'X_4', 'X_6'], dtype='object')
4 most important features:	Index(['X_16', 'X_4', 'X_6', 'X_2'], dtype='object')

Seed: 145

2 most important features:	Index(['X_7', 'X_18'], dtype='object')
3 most important features:	Index(['X_7', 'X_18', 'X_15'], dtype='object')
4 most important features:	Index(['X_7', 'X_18', 'X_15', 'X_3'], dtype='object')

Seed: 146

2 most important features:	Index(['X_1', 'X_7'], dtype='object')
3 most important features:	Index(['X_1', 'X_7', 'X_10'], dtype='object')
4 most important features:	Index(['X_1', 'X_7', 'X_10', 'X_2'], dtype='object')

Seed: 147

2 most important features:	Index(['X_9', 'X_13'], dtype='object')
3 most important features:	Index(['X_9', 'X_13', 'X_16'], dtype='object')
4 most important features:	Index(['X_9', 'X_13', 'X_16', 'X_6'], dtype='object')

Seed: 148

2 most important features:	Index(['X_13', 'X_9'], dtype='object')
3 most 

...

2 most important features:	Index(['X_3', 'X_12'], dtype='object')
3 most important features:	Index(['X_3', 'X_12', 'X_6'], dtype='object')
4 most important features:	Index(['X_3', 'X_12', 'X_6', 'X_1'], dtype='object')

Seed: 829

2 most important features:	Index(['X_9', 'X_10'], dtype='object')
3 most important features:	Index(['X_9', 'X_10', 'X_4'], dtype='object')
4 most important features:	Index(['X_9', 'X_10', 'X_4', 'X_3'], dtype='object')

Seed: 830

2 most important features:	Index(['X_16', 'X_4'], dtype='object')
3 most important features:	Index(['X_16', 'X_4', 'X_9'], dtype='object')
4 most important features:	Index(['X_16', 'X_4', 'X_9', 'X_3'], dtype='object')

Seed: 831

2 most important features:	Index(['X_12', 'X_1'], dtype='object')
3 most important features:	Index(['X_12', 'X_1', 'X_7'], dtype='object')
4 most important features:	Index(['X_12', 'X_1', 'X_7', 'X_9'], dtype='object')

Seed: 832

2 most important features:	Index(['X_18', 'X_6'], dtype='object')
3 most imp

2 most important features:	Index(['X_15', 'X_7'], dtype='object')
3 most important features:	Index(['X_15', 'X_7', 'X_16'], dtype='object')
4 most important features:	Index(['X_15', 'X_7', 'X_16', 'X_13'], dtype='object')

Seed: 865

2 most important features:	Index(['X_18', 'X_6'], dtype='object')
3 most important features:	Index(['X_18', 'X_6', 'X_12'], dtype='object')
4 most important features:	Index(['X_18', 'X_6', 'X_12', 'X_2'], dtype='object')

Seed: 866

2 most important features:	Index(['X_9', 'X_18'], dtype='object')
3 most important features:	Index(['X_9', 'X_18', 'X_4'], dtype='object')
4 most important features:	Index(['X_9', 'X_18', 'X_4', 'X_15'], dtype='object')

Seed: 867

2 most important features:	Index(['X_12', 'X_4'], dtype='object')
3 most important features:	Index(['X_12', 'X_4', 'X_2'], dtype='object')
4 most important features:	Index(['X_12', 'X_4', 'X_2', 'X_16'], dtype='object')

Seed: 868

2 most important features:	Index(['X_10', 'X_16'], dtype='object')
3 

2 most important features:	Index(['X_10', 'X_9'], dtype='object')
3 most important features:	Index(['X_10', 'X_9', 'X_13'], dtype='object')
4 most important features:	Index(['X_10', 'X_9', 'X_13', 'X_3'], dtype='object')

Seed: 901

2 most important features:	Index(['X_7', 'X_10'], dtype='object')
3 most important features:	Index(['X_7', 'X_10', 'X_18'], dtype='object')
4 most important features:	Index(['X_7', 'X_10', 'X_18', 'X_3'], dtype='object')

Seed: 902

2 most important features:	Index(['X_2', 'X_7'], dtype='object')
3 most important features:	Index(['X_2', 'X_7', 'X_12'], dtype='object')
4 most important features:	Index(['X_2', 'X_7', 'X_12', 'X_9'], dtype='object')

Seed: 903

2 most important features:	Index(['X_9', 'X_2'], dtype='object')
3 most important features:	Index(['X_9', 'X_2', 'X_3'], dtype='object')
4 most important features:	Index(['X_9', 'X_2', 'X_3', 'X_4'], dtype='object')

Seed: 904

2 most important features:	Index(['X_10', 'X_2'], dtype='object')
3 most imp

2 most important features:	Index(['X_2', 'X_16'], dtype='object')
3 most important features:	Index(['X_2', 'X_16', 'X_18'], dtype='object')
4 most important features:	Index(['X_2', 'X_16', 'X_18', 'X_12'], dtype='object')

Seed: 937

2 most important features:	Index(['X_6', 'X_3'], dtype='object')
3 most important features:	Index(['X_6', 'X_3', 'X_9'], dtype='object')
4 most important features:	Index(['X_6', 'X_3', 'X_9', 'X_10'], dtype='object')

Seed: 938

2 most important features:	Index(['X_3', 'X_9'], dtype='object')
3 most important features:	Index(['X_3', 'X_9', 'X_1'], dtype='object')
4 most important features:	Index(['X_3', 'X_9', 'X_1', 'X_10'], dtype='object')

Seed: 939

2 most important features:	Index(['X_15', 'X_2'], dtype='object')
3 most important features:	Index(['X_15', 'X_2', 'X_16'], dtype='object')
4 most important features:	Index(['X_15', 'X_2', 'X_16', 'X_6'], dtype='object')

Seed: 940

2 most important features:	Index(['X_10', 'X_12'], dtype='object')
3 most i

2 most important features:	Index(['X_9', 'X_4'], dtype='object')
3 most important features:	Index(['X_9', 'X_4', 'X_16'], dtype='object')
4 most important features:	Index(['X_9', 'X_4', 'X_16', 'X_2'], dtype='object')

Seed: 973

2 most important features:	Index(['X_4', 'X_1'], dtype='object')
3 most important features:	Index(['X_4', 'X_1', 'X_2'], dtype='object')
4 most important features:	Index(['X_4', 'X_1', 'X_2', 'X_18'], dtype='object')

Seed: 974

2 most important features:	Index(['X_16', 'X_18'], dtype='object')
3 most important features:	Index(['X_16', 'X_18', 'X_3'], dtype='object')
4 most important features:	Index(['X_16', 'X_18', 'X_3', 'X_1'], dtype='object')

Seed: 975

2 most important features:	Index(['X_1', 'X_10'], dtype='object')
3 most important features:	Index(['X_1', 'X_10', 'X_7'], dtype='object')
4 most important features:	Index(['X_1', 'X_10', 'X_7', 'X_9'], dtype='object')

Seed: 976

2 most important features:	Index(['X_12', 'X_3'], dtype='object')
3 most imp

In [11]:
important_features_extra = feature_imp_stats(X_ss, y,
                                             n_feats_list = [2, 3, 4], n_runs = 1000, kind="extra")


Seed: 0

2 most important features:	Index(['X_12', 'X_18'], dtype='object')
3 most important features:	Index(['X_12', 'X_18', 'X_15'], dtype='object')
4 most important features:	Index(['X_12', 'X_18', 'X_15', 'X_9'], dtype='object')

Seed: 1

2 most important features:	Index(['X_12', 'X_9'], dtype='object')
3 most important features:	Index(['X_12', 'X_9', 'X_2'], dtype='object')
4 most important features:	Index(['X_12', 'X_9', 'X_2', 'X_10'], dtype='object')

Seed: 2

2 most important features:	Index(['X_15', 'X_9'], dtype='object')
3 most important features:	Index(['X_15', 'X_9', 'X_12'], dtype='object')
4 most important features:	Index(['X_15', 'X_9', 'X_12', 'X_7'], dtype='object')

Seed: 3

2 most important features:	Index(['X_15', 'X_16'], dtype='object')
3 most important features:	Index(['X_15', 'X_16', 'X_18'], dtype='object')
4 most important features:	Index(['X_15', 'X_16', 'X_18', 'X_4'], dtype='object')

Seed: 4

2 most important features:	Index(['X_12', 'X_10'], dtype='obj

2 most important features:	Index(['X_15', 'X_12'], dtype='object')
3 most important features:	Index(['X_15', 'X_12', 'X_2'], dtype='object')
4 most important features:	Index(['X_15', 'X_12', 'X_2', 'X_10'], dtype='object')

Seed: 37

2 most important features:	Index(['X_12', 'X_15'], dtype='object')
3 most important features:	Index(['X_12', 'X_15', 'X_18'], dtype='object')
4 most important features:	Index(['X_12', 'X_15', 'X_18', 'X_1'], dtype='object')

Seed: 38

2 most important features:	Index(['X_12', 'X_9'], dtype='object')
3 most important features:	Index(['X_12', 'X_9', 'X_1'], dtype='object')
4 most important features:	Index(['X_12', 'X_9', 'X_1', 'X_15'], dtype='object')

Seed: 39

2 most important features:	Index(['X_15', 'X_7'], dtype='object')
3 most important features:	Index(['X_15', 'X_7', 'X_18'], dtype='object')
4 most important features:	Index(['X_15', 'X_7', 'X_18', 'X_9'], dtype='object')

Seed: 40

2 most important features:	Index(['X_12', 'X_18'], dtype='object')
3

2 most important features:	Index(['X_15', 'X_18'], dtype='object')
3 most important features:	Index(['X_15', 'X_18', 'X_9'], dtype='object')
4 most important features:	Index(['X_15', 'X_18', 'X_9', 'X_12'], dtype='object')

Seed: 73

2 most important features:	Index(['X_13', 'X_15'], dtype='object')
3 most important features:	Index(['X_13', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_13', 'X_15', 'X_9', 'X_12'], dtype='object')

Seed: 74

2 most important features:	Index(['X_15', 'X_18'], dtype='object')
3 most important features:	Index(['X_15', 'X_18', 'X_9'], dtype='object')
4 most important features:	Index(['X_15', 'X_18', 'X_9', 'X_2'], dtype='object')

Seed: 75

2 most important features:	Index(['X_9', 'X_15'], dtype='object')
3 most important features:	Index(['X_9', 'X_15', 'X_18'], dtype='object')
4 most important features:	Index(['X_9', 'X_15', 'X_18', 'X_6'], dtype='object')

Seed: 76

2 most important features:	Index(['X_18', 'X_12'], dtype='object')


2 most important features:	Index(['X_18', 'X_6'], dtype='object')
3 most important features:	Index(['X_18', 'X_6', 'X_15'], dtype='object')
4 most important features:	Index(['X_18', 'X_6', 'X_15', 'X_12'], dtype='object')

Seed: 109

2 most important features:	Index(['X_15', 'X_18'], dtype='object')
3 most important features:	Index(['X_15', 'X_18', 'X_12'], dtype='object')
4 most important features:	Index(['X_15', 'X_18', 'X_12', 'X_4'], dtype='object')

Seed: 110

2 most important features:	Index(['X_15', 'X_9'], dtype='object')
3 most important features:	Index(['X_15', 'X_9', 'X_12'], dtype='object')
4 most important features:	Index(['X_15', 'X_9', 'X_12', 'X_18'], dtype='object')

Seed: 111

2 most important features:	Index(['X_9', 'X_15'], dtype='object')
3 most important features:	Index(['X_9', 'X_15', 'X_10'], dtype='object')
4 most important features:	Index(['X_9', 'X_15', 'X_10', 'X_6'], dtype='object')

Seed: 112

2 most important features:	Index(['X_15', 'X_9'], dtype='object

2 most important features:	Index(['X_18', 'X_15'], dtype='object')
3 most important features:	Index(['X_18', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_18', 'X_15', 'X_9', 'X_6'], dtype='object')

Seed: 144

2 most important features:	Index(['X_18', 'X_12'], dtype='object')
3 most important features:	Index(['X_18', 'X_12', 'X_6'], dtype='object')
4 most important features:	Index(['X_18', 'X_12', 'X_6', 'X_16'], dtype='object')

Seed: 145

2 most important features:	Index(['X_15', 'X_9'], dtype='object')
3 most important features:	Index(['X_15', 'X_9', 'X_18'], dtype='object')
4 most important features:	Index(['X_15', 'X_9', 'X_18', 'X_7'], dtype='object')

Seed: 146

2 most important features:	Index(['X_4', 'X_15'], dtype='object')
3 most important features:	Index(['X_4', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_4', 'X_15', 'X_9', 'X_18'], dtype='object')

Seed: 147

2 most important features:	Index(['X_15', 'X_18'], dtype='object')

...

2 most important features:	Index(['X_10', 'X_15'], dtype='object')
3 most important features:	Index(['X_10', 'X_15', 'X_7'], dtype='object')
4 most important features:	Index(['X_10', 'X_15', 'X_7', 'X_6'], dtype='object')

Seed: 847

2 most important features:	Index(['X_12', 'X_15'], dtype='object')
3 most important features:	Index(['X_12', 'X_15', 'X_18'], dtype='object')
4 most important features:	Index(['X_12', 'X_15', 'X_18', 'X_9'], dtype='object')

Seed: 848

2 most important features:	Index(['X_12', 'X_15'], dtype='object')
3 most important features:	Index(['X_12', 'X_15', 'X_18'], dtype='object')
4 most important features:	Index(['X_12', 'X_15', 'X_18', 'X_9'], dtype='object')

Seed: 849

2 most important features:	Index(['X_18', 'X_12'], dtype='object')
3 most important features:	Index(['X_18', 'X_12', 'X_7'], dtype='object')
4 most important features:	Index(['X_18', 'X_12', 'X_7', 'X_9'], dtype='object')

Seed: 850

2 most important features:	Index(['X_9', 'X_18'], dtype='obj

2 most important features:	Index(['X_15', 'X_1'], dtype='object')
3 most important features:	Index(['X_15', 'X_1', 'X_2'], dtype='object')
4 most important features:	Index(['X_15', 'X_1', 'X_2', 'X_12'], dtype='object')

Seed: 882

2 most important features:	Index(['X_12', 'X_15'], dtype='object')
3 most important features:	Index(['X_12', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_12', 'X_15', 'X_9', 'X_6'], dtype='object')

Seed: 883

2 most important features:	Index(['X_18', 'X_7'], dtype='object')
3 most important features:	Index(['X_18', 'X_7', 'X_9'], dtype='object')
4 most important features:	Index(['X_18', 'X_7', 'X_9', 'X_15'], dtype='object')

Seed: 884

2 most important features:	Index(['X_6', 'X_12'], dtype='object')
3 most important features:	Index(['X_6', 'X_12', 'X_15'], dtype='object')
4 most important features:	Index(['X_6', 'X_12', 'X_15', 'X_9'], dtype='object')

Seed: 885

2 most important features:	Index(['X_18', 'X_9'], dtype='object')
3 m

2 most important features:	Index(['X_12', 'X_18'], dtype='object')
3 most important features:	Index(['X_12', 'X_18', 'X_9'], dtype='object')
4 most important features:	Index(['X_12', 'X_18', 'X_9', 'X_15'], dtype='object')

Seed: 917

2 most important features:	Index(['X_18', 'X_2'], dtype='object')
3 most important features:	Index(['X_18', 'X_2', 'X_15'], dtype='object')
4 most important features:	Index(['X_18', 'X_2', 'X_15', 'X_12'], dtype='object')

Seed: 918

2 most important features:	Index(['X_18', 'X_12'], dtype='object')
3 most important features:	Index(['X_18', 'X_12', 'X_1'], dtype='object')
4 most important features:	Index(['X_18', 'X_12', 'X_1', 'X_10'], dtype='object')

Seed: 919

2 most important features:	Index(['X_18', 'X_15'], dtype='object')
3 most important features:	Index(['X_18', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_18', 'X_15', 'X_9', 'X_1'], dtype='object')

Seed: 920

2 most important features:	Index(['X_15', 'X_18'], dtype='obje

2 most important features:	Index(['X_9', 'X_18'], dtype='object')
3 most important features:	Index(['X_9', 'X_18', 'X_12'], dtype='object')
4 most important features:	Index(['X_9', 'X_18', 'X_12', 'X_6'], dtype='object')

Seed: 952

2 most important features:	Index(['X_15', 'X_1'], dtype='object')
3 most important features:	Index(['X_15', 'X_1', 'X_18'], dtype='object')
4 most important features:	Index(['X_15', 'X_1', 'X_18', 'X_12'], dtype='object')

Seed: 953

2 most important features:	Index(['X_1', 'X_12'], dtype='object')
3 most important features:	Index(['X_1', 'X_12', 'X_18'], dtype='object')
4 most important features:	Index(['X_1', 'X_12', 'X_18', 'X_2'], dtype='object')

Seed: 954

2 most important features:	Index(['X_15', 'X_12'], dtype='object')
3 most important features:	Index(['X_15', 'X_12', 'X_18'], dtype='object')
4 most important features:	Index(['X_15', 'X_12', 'X_18', 'X_1'], dtype='object')

Seed: 955

2 most important features:	Index(['X_6', 'X_18'], dtype='object'

2 most important features:	Index(['X_18', 'X_15'], dtype='object')
3 most important features:	Index(['X_18', 'X_15', 'X_9'], dtype='object')
4 most important features:	Index(['X_18', 'X_15', 'X_9', 'X_16'], dtype='object')

Seed: 987

2 most important features:	Index(['X_9', 'X_15'], dtype='object')
3 most important features:	Index(['X_9', 'X_15', 'X_18'], dtype='object')
4 most important features:	Index(['X_9', 'X_15', 'X_18', 'X_16'], dtype='object')

Seed: 988

2 most important features:	Index(['X_15', 'X_18'], dtype='object')
3 most important features:	Index(['X_15', 'X_18', 'X_12'], dtype='object')
4 most important features:	Index(['X_15', 'X_18', 'X_12', 'X_4'], dtype='object')

Seed: 989

2 most important features:	Index(['X_9', 'X_18'], dtype='object')
3 most important features:	Index(['X_9', 'X_18', 'X_12'], dtype='object')
4 most important features:	Index(['X_9', 'X_18', 'X_12', 'X_15'], dtype='object')

Seed: 990

2 most important features:	Index(['X_18', 'X_9'], dtype='obje

In [12]:
important_features_random

{2: {'X_10': 177,
  'X_1': 166,
  'X_4': 165,
  'X_13': 159,
  'X_9': 158,
  'X_3': 157,
  'X_2': 154,
  'X_7': 151,
  'X_15': 148,
  'X_12': 144,
  'X_6': 142,
  'X_16': 142,
  'X_18': 137},
 3: {'X_4': 256,
  'X_13': 246,
  'X_1': 243,
  'X_10': 243,
  'X_3': 236,
  'X_7': 232,
  'X_9': 228,
  'X_18': 224,
  'X_12': 221,
  'X_2': 219,
  'X_16': 218,
  'X_6': 217,
  'X_15': 217},
 4: {'X_10': 336,
  'X_4': 325,
  'X_12': 319,
  'X_3': 314,
  'X_7': 310,
  'X_2': 308,
  'X_1': 308,
  'X_13': 306,
  'X_16': 304,
  'X_15': 299,
  'X_9': 298,
  'X_18': 295,
  'X_6': 278}}

In [13]:
important_features_extra

{2: {'X_18': 575,
  'X_15': 562,
  'X_12': 418,
  'X_9': 228,
  'X_6': 69,
  'X_1': 44,
  'X_4': 23,
  'X_7': 19,
  'X_2': 18,
  'X_13': 15,
  'X_16': 14,
  'X_10': 12,
  'X_3': 3},
 3: {'X_18': 742,
  'X_15': 714,
  'X_12': 620,
  'X_9': 382,
  'X_6': 138,
  'X_1': 101,
  'X_4': 58,
  'X_7': 52,
  'X_16': 50,
  'X_2': 44,
  'X_13': 44,
  'X_10': 37,
  'X_3': 18},
 4: {'X_18': 849,
  'X_15': 844,
  'X_12': 748,
  'X_9': 532,
  'X_6': 224,
  'X_1': 174,
  'X_4': 122,
  'X_7': 113,
  'X_2': 98,
  'X_13': 93,
  'X_10': 88,
  'X_16': 85,
  'X_3': 30}}

After analysing the result above (looking at key "4"), it's clear that the most important features are:

In [ ]:
most_important_features_extra = "X_18 X_15 X_12 X_9".split()

most_important_features_random = "X_10 X_4 X_12 X_3".split()

In [ ]:
features = X_ss.columns.tolist()

In [ ]:
idxs_extra = [features.index(item) for item in most_important_features_extra]

idxs_random = [features.index(item) for item in most_important_features_random]

In [ ]:
actual_features = [x 
                   for sublist in [f"(X, {i})#(Y, {i})#(Z, {i})".split("#") for i in range(1, 7)]
                   for x in sublist]

In [ ]:
[actual_features[i] for i in idxs_extra]

In [ ]:
[actual_features[i] for i in idxs_random]

New: let's try both!

In [ ]:
most_important_features_dict = {"extra": most_important_features_extra.copy(),
                                "random": most_important_features_random.copy()}

In [ ]:
most_important_features_dict

__________

In [ ]:
import os

def model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "kmeans"):   
    
    for strategy, most_important_features in most_important_features_dict.items():

        print()
        print("#"*80)
        print(f"Feature selection strategy: {strategy}".center(80))
        print("#"*80)
        print()

        for n_bins, n_feats in combs:

            ###################################################

            file_specs = f"{n_feats}_{n_bins}_desambiguation_{how}_fi_{strategy}_discretization_{kbins_strategy}_refit_discret"

            ###################################################

            start = time.time()

            print("\n")
            print("="*50)
            print(f"n_bins : {n_bins}".center(50))
            print(f"n_feats : {n_feats}".center(50))
            print("="*50)
            print("\n")

            ###################################################

            X_fi = X_ss.loc[:, most_important_features[:n_feats]].copy()

            # creating "n_bins" uniform bins to discretize continuous features
            bin_fi = KBinsDiscretizer(n_bins=n_bins, 
                                      encode='ordinal',
                                      strategy=kbins_strategy).fit(X_fi)

            # transforming feature importance-reduced dataset
            aux = pd.DataFrame(bin_fi.transform(X_fi)).astype("str")
            aux.columns = [f"Xb_{i+1}" for i in range(aux.shape[1])]

            # creating dummy variables
            aux = pd.get_dummies(aux, drop_first=True)

            df_fi_bin = pd.concat([aux, y], axis=1)

            ###################################################

            X_fi_bin = df_fi_bin.drop(columns="y")
            y_fi_bin = df_fi_bin["y"]

            ###################################################

            # desambiguation is made within .fit() method!
            qnn_fi_bin = QNN().fit(X_fi_bin, y_fi_bin, how=how)

            ###################################################

            pipe = Pipeline([('knn', KNeighborsClassifier(n_jobs=-1))])

            param_grid = {'knn__n_neighbors': np.arange(1, 100),
                          'knn__weights':['uniform', 'distance'], 
                          'knn__metric':['euclidean', 'hamming']}

            # with verbose=10, info appears in the terminal/prompt!
            knn_fi_bin = GridSearchCV(pipe, param_grid, scoring="f1_weighted", 
                                      cv=10, verbose=10, n_jobs=-1)

            knn_fi_bin.fit(X_fi_bin.values, y_fi_bin)

            print(f"\nBest classical model parameters:\n{knn_fi_bin.best_params_}")

            ###################################################

            print("\n\n")

            for label, df_test in df_test_dic.items():
                
                # ========================================
                
                file_quantum = f"./results/predict_proba_test_{label}_QUANTUM_fi_new_features_{file_specs}.csv"
                file_classical = f"./results/predict_proba_test_{label}_CLASSICAL_fi_new_features_{file_specs}.csv"
                
                # ========================================

                if os.path.isfile(file_quantum):
                    
                    print(f"File with predictions of {label} (QUANTUM) already exists!\n")
                    
                else:
                    
                    print(f"Predicting targets for {label} with feature importance - QUANTUM!\n")

                    X_test = df_test.drop(columns=["g"]).copy()

                    # ========================================
                    
                    # new 01/04/2022: refit ss in test
                    ss_test = StandardScaler()
                    X_test_ss = pd.DataFrame(ss_test.fit_transform(X_test), index=X_test.index, columns=X_test.columns)

                    X_test_fi = X_test_ss.loc[:, most_important_features[:n_feats]].copy()

                    # new 25/03/22: refit bin in test
                    bin_fi_test = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=kbins_strategy)
                    aux = pd.DataFrame(bin_fi_test.fit_transform(X_test_fi)).astype("str")
                    aux.columns = [f"Xb_{i+1}" for i in range(aux.shape[1])]

                    X_test_fi = pd.get_dummies(aux, drop_first=True)

                    # ========================================

                    y_pred_fi, y_proba_fi = qnn_fi_bin.predict(X_test_fi)

                    # ========================================

                    df_result = pd.concat([df_test["g"], 
                                           pd.Series(np.array(y_proba_fi)[:, 1], name="predict_proba_y=1")], 
                                          axis=1)

                    df_result.to_csv(file_quantum, index=False)

                ###################################################    
                # CLASSICAL BENCHMARK
                ###################################################    

                if os.path.isfile(file_classical):
                    
                    print(f"File with predictions of {label} (CLASSICAL) already exists!\n")
                    
                else:
                    print(f"Predicting targets for {label} with feature importance - CLASSICAL!")

                    # ========================================

                    X_test_fi = X_test_fi.to_numpy()

                    train_dims = X_fi_bin.shape[1]
                    test_dims = X_test_fi.shape[1]

                    # in this case, i'll just try some padding
                    if test_dims < train_dims:

                        X_test_fi = np.pad(X_test_fi, 
                                            [(0, 0), (0, train_dims - test_dims)],
                                            mode='constant', 
                                            constant_values=0).copy()

                    # in this case, there's not much to be done...
                    # but i think this won't happen. let's cover it anyway, though
                    elif test_dims > train_dims:

                        error_str = "Dimensionality mismatch!"
                        error_str += f"\nTraining data has {X_fi_bin.shape[1]} dimensions"
                        error_str += f"\nAnd test dat has {X_test_fi.shape[1]} dimensions!"
                        error_str += f"\nPlease check pre-processing!"

                        raise ValueError(error_str)

                    # ========================================

                    y_proba_fi = knn_fi_bin.predict_proba(X_test_fi)

                    # ========================================

                    df_result = pd.concat([df_test["g"], 
                                           pd.Series(np.array(y_proba_fi)[:, 1], name="predict_proba_y=1")], 
                                          axis=1)

                    df_result.to_csv(file_classical, index=False)

                    print("\n")
                    print("~"*30)
                    print("\n")

            ###################################################

            stop = time.time()
            duration = time.strftime("%H:%M:%S", time.gmtime(stop-start))

            print(f"\n----> Model execution time: {duration}")

Combinations of number of features and number of bins to be tested

In [ ]:
n_bins_list = [2, 3]
n_feats_list = [2, 3, 4]

combs = [(2, 2), 
         (2, 3), (2, 4),
         (3, 3), (3, 4)]

### No desambiguation

In [ ]:
how = None

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "quantile")

### Desambiguation with minority class

In [ ]:
how = "minor"

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "quantile")

### Desambiguation with majority class

In [ ]:
how = "major"

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "quantile")

### Desambiguation with first appearance class

In [ ]:
how = "first"

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "kmeans")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "uniform")

In [ ]:
model_pipeline(X_ss, df_test_dic, how, most_important_features_dict, combs, kbins_strategy = "quantile")

_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______
_______

### Study of the quantum circuit

In [ ]:
qc0 = qnn_fi_bin.construct_circuit(X_test, k=42)

In [ ]:
qc0.draw("mpl")

In [ ]:
qc0.decompose().decompose().draw("mpl")

In [ ]:
# 20 qubits backend
from qiskit.test.mock import FakeTokyo

from qiskit import transpile
from qiskit.visualization import plot_circuit_layout

##################################

for opt_level in range(4):
    
    transp_circ = transpile(qc0, backend, optimization_level=opt_level)
    
    print(f'Optimization Level {opt_level}')
    print(f'Depth: {transp_circ.depth()}')
    print(f'Gate counts: {transp_circ.count_ops()}')
    print(f'Total number of gates: {sum(transp_circ.count_ops().values())}')
    
    show_figure(plot_circuit_layout(transp_circ, backend))
    
#     show_figure(transp_circ.draw("mpl"))
    
    print("="*50)

Important!! As described [here](https://qiskit.org/documentation/apidoc/transpiler.html),

> "However, as with many important things in life, finding the optimal SWAP mapping is hard. In fact it is in a class of problems called NP-Hard, and is thus prohibitively expensive to compute for all but the smallest quantum devices and input circuits. To get around this, by default Qiskit uses a stochastic heuristic algorithm called `Qiskit.transpiler.passes.StochasticSwap` to compute a good, but not necessarily minimal SWAP count. The use of a stochastic method means the circuits generated by `Qiskit.compiler.transpile()` (or `Qiskit.execute()` that calls transpile internally) are not guaranteed to be the same over repeated runs. Indeed, running the same circuit repeatedly will in general result in a distribution of circuit depths and gate counts at the output."

______

For testing circuit with actual backend noise model: https://qiskit.org/documentation/tutorials/simulators/2_device_noise_simulation.html